<a href="https://colab.research.google.com/github/RaxFord1/RL-Projects/blob/master/FrozenLakeV0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To save video, we need X screen, suddenly colab is a cloud-service and we don't have screen at all, so we need to simulate it with xvfb lib

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

Creating virtual screen

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

**Importing all needs `gym, pytorch, numpy`**

In [0]:
import random
import gym
import gym.spaces
import gym.wrappers
import gym.envs.toy_text.frozen_lake
from collections import namedtuple
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [0]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [0]:
env = gym.envs.toy_text.frozen_lake.FrozenLakeEnv(is_slippery=False)
env = gym.wrappers.TimeLimit(env, max_episode_steps=100)
env = DiscreteOneHotWrapper(env)

In [6]:
env.observation_space.shape[0]

16

In [7]:
env.action_space.n

4

In [0]:
HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.99

In [0]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


In [0]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [0]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [0]:
def filter_batch(batch, percentile):
    disc_rewards = list(map(lambda s: s.reward * (GAMMA ** len(s.steps)), batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation, example.steps))
            train_act.extend(map(lambda step: step.action, example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound

In [13]:
if __name__ == "__main__":
    random.seed(12345)

    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.001)

    full_batch = []
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
        full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
        if not full_batch:
            continue
        obs_v = torch.FloatTensor(obs)
        acts_v = torch.LongTensor(acts)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.3f, reward_bound=%.3f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch)))
        if reward_mean > 0.99:
            print("Solved!")
            break


1: loss=1.377, reward_mean=0.020, reward_bound=0.000, batch=2
2: loss=1.370, reward_mean=0.000, reward_bound=0.000, batch=2
3: loss=1.368, reward_mean=0.040, reward_bound=0.000, batch=6
4: loss=1.357, reward_mean=0.020, reward_bound=0.000, batch=8
5: loss=1.353, reward_mean=0.030, reward_bound=0.000, batch=11
6: loss=1.347, reward_mean=0.020, reward_bound=0.000, batch=13
7: loss=1.350, reward_mean=0.050, reward_bound=0.000, batch=18
8: loss=1.344, reward_mean=0.040, reward_bound=0.000, batch=22
9: loss=1.339, reward_mean=0.050, reward_bound=0.000, batch=27
10: loss=1.334, reward_mean=0.040, reward_bound=0.000, batch=31
11: loss=1.328, reward_mean=0.040, reward_bound=0.000, batch=35
12: loss=1.322, reward_mean=0.010, reward_bound=0.000, batch=36
13: loss=1.323, reward_mean=0.040, reward_bound=0.000, batch=40
14: loss=1.317, reward_mean=0.030, reward_bound=0.000, batch=43
15: loss=1.313, reward_mean=0.030, reward_bound=0.000, batch=46
16: loss=1.307, reward_mean=0.050, reward_bound=0.000

In [0]:
#env = gym.wrappers.Monitor(env, directory="mon", force=True)

In [0]:
def play_n_times(env, net, n):
  history = list()
  
  for i in range(n):
    rewards = 0
    obs = env.reset()
    done = False
    
    while not done:
      #NN gets only tensor-like information
      obs = torch.FloatTensor([obs])
      #getting predictions from our NN
      probs = net(obs)
      #using activation layer on results
      sm = nn.Softmax(dim=1)
      act_probs_v = sm(probs)
      #converting to numpy data-type
      act_probs_v.data.numpy()[0]
      act_probs = act_probs_v.data.numpy()[0]
      #getting index of highest prediction
      act_probs.max()
      action = np.where(act_probs == act_probs.max())[0][0]
      next_obs, reward, is_done, _ = env.step(action)
      rewards += reward
      done = is_done
      obs = next_obs
      
    history.append(rewards)
    
  return history

In [0]:
history = play_n_times(env, net, 100)

In [17]:
history[:10]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [0]:
history = np.array(history)

In [19]:
print("Max = %d, Min = %d, Mean = %d"%(history.max(), history.min(), history.mean()))

Max = 1, Min = 1, Mean = 1
